In [1]:
%matplotlib inline
from pyqubo import Array, Placeholder, solve_qubo, Constraint, Sum
from dwave.system.samplers import DWaveSampler
from dwave.system.composites import EmbeddingComposite
from dwave_qbsolv import QBSolv

import pandas as pd
import numpy as np

import optuna

### Hamiltonian

N-Queen問題のハミルトニアン、　列column、行row、斜めdiagonal

$$H = \sum_{col}\left(\sum_{i \in col} x_i - 1\right)^2 + \sum_{row}\left(\sum_{i \in row} x_i - 1\right)^2 + \sum_{diag}\left\{\left(\sum_{i \in diag} x_i\right)\left(\sum_{i \in diag} x_i - 1\right)\right\}$$


In [2]:
# N-Queenのマスの数
N = 8

#pyQUBOで0、１の変数をNxN正方マス分つくる
q = Array.create('q', (N*N), 'BINARY')

q_shape = np.reshape(q,(N,N))
#print(q_shape)


In [3]:
#コストをPyQUBOで記述

#row
row_const = 0.0
for row in range(N):
    row_const += (sum(i for i in q_shape[row, :]) -1)**2


#colum
col_const = 0.0
for col in range(N):
    col_const += (sum(i for i in q_shape[:, col]) -1)**2
    #print(col_const)
    #print('---')

#斜め
# ＼の方
diag_const = 0.0
xi = 0.0
xi_1 = 0.0
for k in range(-N+2,N-1):
    xi += (sum(i for i in np.diag(q_shape,k=k)))
    xi_1 += (sum(i for i in np.diag(q_shape,k=k))) -1 
    diag_const += xi * xi_1


#斜め
# /の方のために入れ替えて, ＼ をする。
diag_const_f = 0.0
xi = 0.0
xi_1 = 0.0
for k in range(-N+2,N-1):
    xi += (sum(i for i in np.diag(np.fliplr(q_shape),k=k)))
    xi_1 += (sum(i for i in np.diag(np.fliplr(q_shape),k=k))) -1 
    diag_const_f += xi * xi_1


# Hyper parameter and Hamiltonian is using Placeholder
alpha = Placeholder("alpha")
beta = Placeholder("beta")
gamma = Placeholder("gamma")
H = alpha * col_const + beta * row_const  + gamma *(diag_const + diag_const_f)

In [4]:
def calc_dwave(param_a,param_b,param_c,param_num_reads,param_index_label,param_qv):
    # 制約項のペナルティウェイト
    # alpha = param_a
    # beta = param_b
    # gamma = param_c
    # num_reads = param_num_reads
    # index_label= param_index_label #True, False
    # pv = param_qv #True, False

    #パラメータ
    feed_dict = {'alpha': param_a, 'beta': param_b, 'gamma': param_c}

    #QUBOのコンパイル
    model = H.compile()
    qubo, offset = model.to_qubo(feed_dict=feed_dict, index_label= param_index_label)

    # Solve QUBO model by D-Wave
    # ---------------------------
    endpoint = "https://cloud.dwavesys.com/sapi"
    token = "CV19-" #<-- your token 
    solver_name = "DW_2000Q_5"
    
    sampler = EmbeddingComposite(DWaveSampler(endpoint=endpoint,token=token,solver=solver_name))

    if param_qv == True: 
        response = QBSolv().sample_qubo(qubo, num_reads = param_num_reads) 
    else:
        response = sampler.sample_qubo(qubo, num_reads = param_num_reads)

    return response

In [5]:
def solution_check(res_pd_p):
#　n-queenのチェックプログラム

    res_pd_p = res_pd_p

    # 行数を確認
    for kk in range(len(res_pd_p.index)):
        # 行のスライス 0:1, q[0]..q[N*N]　切り取り
        res_pd_NN = res_pd_p.iloc[kk:kk+1, 0:N*N]
        # q[0]..q[N*N]に並べ替え
        res_pd_NNsort = res_pd_NN.loc[:, list(map(lambda x: "q[{}]".format(x), range(0,N*N)))]

        #pandas から arrayへ
        res_pd_NNsortarray = res_pd_NNsort.values
        res_pd_NN_shape = np.reshape(res_pd_NNsortarray ,(N,N))

        #print(res_pd_NN_shape)

        #ここから、判定　もし　満たしてなければ、 param_x_ng へフラグを立てる

        #行-　の足し算 の判定
        for i in np.sum(res_pd_NN_shape, axis=1):
            if  i >= 2: 
                res_pd_p.at[kk,'param_a_ng'] = 1
            else:
                pass

        #列|　の足し算
        for i in np.sum(res_pd_NN_shape, axis=0):
            if  i >= 2: 
                res_pd_p.at[kk,'param_b_ng'] = 1
            else:
                pass

        #　斜め/
        for k in range(-N+2,N-1):
            i = np.sum(np.diag(res_pd_NN_shape,k=k))
            if  i >= 2: 
                res_pd_p.at[kk,'param_c_ng'] = 1
            else:
                pass

        #　斜め\
        for k in range(-N+2,N-1):
            i = np.sum(np.diag(np.fliplr(res_pd_NN_shape),k=k))
            if  i >= 2: 
                res_pd_p.at[kk,'param_c_ng'] = 1
            else:
                pass

    #or で param a, b, cにNGあったらparam_ngにまとめる
    res_pd_p['param_ng'] = ((res_pd_p['param_a_ng'] == 1) | (res_pd_p['param_b_ng'] == 1) | (res_pd_p   ['param_c_ng'] == 1))

    #ture, false に *1すると　　0,1になる。
    res_pd_p['occurrences_prob'] = res_pd_p['num_occurrences'] * res_pd_p['param_ng'] * 1

    #全部でどのくらいfissible non-fisibileがあるかカウント　num_occurrences
    ##res_pd_p["num_occurrences"].sum(axis=0)

    #失敗率
    o_prob = res_pd_p["occurrences_prob"].sum(axis=0)/res_pd_p["num_occurrences"].sum(axis=0)

    return o_prob


In [6]:
def calc_main(xc):
    alpha = 1.0
    beta = 1.0   
    gamma = xc

    #
    # response = calc_dwave(alpha, beta, gamma ,10,False,False)
    #  (alpha, beta, gamma ,計算回数=10,pyqubo index label=False, D-wave or QBSolv use ex: True = D-Wave, False=QBSolve)
    #
    response = calc_dwave(alpha, beta, gamma ,10,False,True)
    #結果のPandas化
    res_pd = response.to_pandas_dataframe()
    #行、列、斜めで、満たさない場合がある場合にフラグと立てる場所の項目を作成
    res_pd_p = res_pd.assign(param_a_ng = 0, param_b_ng = 0, param_c_ng = 0)
    score_m = solution_check(res_pd_p)

    return score_m

In [7]:
# 失敗率でoptunaでパラメータチューニングしてみる。

def objective(trial):
    x = trial.suggest_uniform('x', 0,1)

    score=calc_main(x)

    print('x: %1.3f, score: %1.3f' % (x, score))
    return score

In [8]:
study = optuna.create_study()
study.optimize(objective, n_trials=10)
print("params_{}".format(study.best_params))
print("value_{}".format(study.best_value))

[I 2020-04-02 14:13:09,287] Finished trial#0 resulted in value: 0.014925373134328358. Current best value is 0.014925373134328358 with parameters: {'x': 0.3716248996269532}.
x: 0.372, score: 0.015
[I 2020-04-02 14:13:11,342] Finished trial#1 resulted in value: 0.918918918918919. Current best value is 0.014925373134328358 with parameters: {'x': 0.3716248996269532}.
x: 0.056, score: 0.919
[I 2020-04-02 14:13:13,497] Finished trial#2 resulted in value: 1.0. Current best value is 0.014925373134328358 with parameters: {'x': 0.3716248996269532}.
x: 0.022, score: 1.000
[I 2020-04-02 14:13:15,488] Finished trial#3 resulted in value: 0.0. Current best value is 0.0 with parameters: {'x': 0.2101892532802433}.
x: 0.210, score: 0.000
[I 2020-04-02 14:13:17,501] Finished trial#4 resulted in value: 0.0. Current best value is 0.0 with parameters: {'x': 0.2101892532802433}.
x: 0.661, score: 0.000
[I 2020-04-02 14:13:19,568] Finished trial#5 resulted in value: 0.0. Current best value is 0.0 with paramete